In [1]:
import sys

!{sys.executable} -m pip install -U --pre "mxnet>=1.7.0b20200713, <2.0.0" -f https://sxjscience.github.io/KDD2020/


Looking in links: https://sxjscience.github.io/KDD2020/
     |████████████████████████████████| 55.0MB 57kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [2]:
!{sys.executable} -m pip install autogluon

     |████████████████████████████████| 624kB 5.7MB/s 
     |████████████████████████████████| 102kB 9.7MB/s 
     |████████████████████████████████| 2.1MB 18.9MB/s 
     |████████████████████████████████| 2.6MB 43.2MB/s 
     |████████████████████████████████| 112kB 43.3MB/s 
     |████████████████████████████████| 28.6MB 116kB/s 
     |████████████████████████████████| 215kB 41.6MB/s 
     |████████████████████████████████| 153kB 38.6MB/s 
     |████████████████████████████████| 819kB 43.3MB/s 
     |████████████████████████████████| 133kB 46.4MB/s 
     |████████████████████████████████| 66.3MB 42kB/s 
     |████████████████████████████████| 890kB 40.5MB/s 
     |████████████████████████████████| 1.8MB 37.9MB/s 
     |████████████████████████████████| 665kB 42.9MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 962kB 36.0MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 3.0MB 38.2MB/s 

In [3]:
!{sys.executable} -m pip install bokeh==2.0.1

     |████████████████████████████████| 8.6MB 5.2MB/s 
  Created wheel for bokeh: filename=bokeh-2.0.1-cp36-none-any.whl size=9080030 sha256=c7d0a350e325dd3428ba7dc7c3fffc8c76c42cadfbd11d7950a5cbc1d99de098
  Stored in directory: /root/.cache/pip/wheels/de/64/d0/e7f70919e19f57a6efceb65cc4edaa0449e707bcdb6e7e61ad
Successfully built bokeh
ERROR: panel 0.9.7 has requirement bokeh>=2.1, but you'll have bokeh 2.0.1 which is incompatible.
  Found existing installation: bokeh 2.1.1
    Uninstalling bokeh-2.1.1:
      Successfully uninstalled bokeh-2.1.1


AFTER RUN PRECEDING PIP INSTALLS (ETC), MUST RESTART RUNTIME (Runtime>>Restart Runtime)


In [1]:
import autogluon as ag
from autogluon import TabularPrediction as task

In [2]:
###READ IN DATA
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style('darkgrid')
plt.style.use('seaborn-poster')

In [3]:
from google.colab import files
uploaded = files.upload()

Saving TemperatureOverTimeByStation.csv to TemperatureOverTimeByStation.csv


In [4]:
import io
data = pd.read_csv(io.BytesIO(uploaded['TemperatureOverTimeByStation.csv']))

In [5]:
data.shape

(33865, 19)

In [6]:
data.head()

,Unnamed: 0,Hourstart,Temp_4847,Temp_94889,Temp_94847,Temp_94815,Temp_14833,Temp_14836,Temp_94860,Temp_4888,Temp_14826,Temp_14840,Temp_14845,Temp_94898,Temp_94894,Temp_14817,Temp_14850,Temp_94849,Temp_14841
0,1,2017-01-01 01:00:00,28.0,19.9,27.0,23.0,23.0,26.1,27.0,28.4,28.0,30.0,30.0,30.9,25.0,22.1,27.0,28.9,25.0
1,2,2017-01-01 02:00:00,26.1,19.0,26.1,25.0,23.0,24.1,24.1,28.4,28.0,25.0,28.0,30.0,24.1,21.6,26.1,26.1,28.0
2,3,2017-01-01 03:00:00,25.0,18.0,23.0,21.0,24.1,23.0,26.1,26.6,25.0,26.1,26.1,28.0,25.0,20.1,26.1,25.0,28.0
3,4,2017-01-01 04:00:00,24.1,19.9,24.1,26.1,24.1,23.0,24.1,24.8,25.0,30.0,25.0,28.0,24.1,19.4,27.0,24.1,28.0
4,5,2017-01-01 05:00:00,23.0,16.0,24.1,25.0,24.1,23.0,26.1,24.8,25.0,30.9,25.0,27.0,24.1,23.0,28.0,24.1,27.0


In [11]:
data=data.drop(['Unnamed: 0'],axis=1)
#Drop column added during R save

In [35]:
sorted_zipped_lists=sorted(zip(data.isna().sum(),list(data))) #sorts zipped lists by first list, which is the # of NA values for a given column

In [46]:
sorted_column_names = [element for _, element in sorted_zipped_lists]
sorted_column_names.remove('Hourstart') #remove Hourstart from list, because we don't need to impute any missing values for this variable

['Temp_94860', 'Temp_94847', 'Temp_14845', 'Temp_94849', 'Temp_14841', 'Temp_94815', 'Temp_14850', 'Temp_14826', 'Temp_14840', 'Temp_14836', 'Temp_14833', 'Temp_4847', 'Temp_14817', 'Temp_4888', 'Temp_94889', 'Temp_94898', 'Temp_94894']


In [50]:
#Loop through each weather station in 'sorted_column_names' (ascending in terms of the N of missing observations, so we leave the largest imputation for the most complete dataset)
#AND impute missing values, then move to next station
for LC in sorted_column_names:
    label_column = LC
    print(label_column)
    nomissings=data.dropna(subset=[label_column]) #drop rows with missing values for the given column
    train = nomissings.sample(frac=0.75, random_state=99)
    # you can't simply split 0.75 and 0.25 without overlapping
    # this code tries to find that train = 75% and test = 25%
    test = nomissings.loc[~nomissings.index.isin(train.index), :]

    predictor = task.fit(train_data=train, tuning_data=test,
                        label=label_column,eval_metric = "r2")

    #performance of best model (or ensemble)
    performance = predictor.evaluate(test)

    #overall results (i.e. all different models)
    results = predictor.fit_summary()

    missers = data.loc[~data.index.isin(nomissings.index), :] 
    #from the *data* df, pull those rows that 
    #weren't included once we dropped rows that were missing values for the given column
    predicted_missingvalues = predictor.predict(missers)

    missers[label_column]=predicted_missingvalues #plug-in interpreted values, obtained from autogluon

    combined=nomissings.append(missers)
    #recombine missers with nonmissers, creating a more robust dataset to model downstream analyses

    data = combined

Temp_94860
Temp_94847
Temp_14845
Temp_94849
Temp_14841
Temp_94815
Temp_14850
Temp_14826
Temp_14840
Temp_14836
Temp_14833
Temp_4847
Temp_14817
Temp_4888
Temp_94889
Temp_94898
Temp_94894


In [51]:
label_column = 'Temp_94860'

In [52]:
nomissings=data.dropna(subset=['Temp_94860']) #drop rows with missing MINIMUM_PAYMENTS values
train = nomissings.sample(frac=0.75, random_state=99)
# you can't simply split 0.75 and 0.25 without overlapping
# this code tries to find that train = 75% and test = 25%
test = nomissings.loc[~nomissings.index.isin(train.index), :]

NumExpr defaulting to 2 threads.


In [55]:
predictor = task.fit(train_data=train, tuning_data=test,
                     label=label_column,eval_metric = "r2")


No output_directory specified. Models will be saved in: AutogluonModels/ag-20201202_175346/
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20201202_175346/
AutoGluon Version:  0.0.14
Train Data Rows:    25396
Train Data Columns: 17
Tuning Data Rows:    8466
Tuning Data Columns: 17
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and label-values can't be converted to int).
	Label info (max, min, mean, stddev): (93.9, -9.9, 50.1799, 19.74923)
	If 'regression' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    12650.44 MB
	Train Data (Original)  Memory Usage: 6.91 MB (0.1% of available memory)
	Inferring data type of each feature bas

[1000]	train_set's l2: 0.666801	train_set's r2: 0.998089	valid_set's l2: 2.15408	valid_set's r2: 0.9944


	0.9946	 = Validation r2 score
	7.59s	 = Training runtime
	0.64s	 = Validation runtime
Fitting model: LightGBMRegressorXT ...


[1000]	train_set's l2: 1.65661	train_set's r2: 0.995526	valid_set's l2: 2.403	valid_set's r2: 0.993779
[2000]	train_set's l2: 1.24319	train_set's r2: 0.996597	valid_set's l2: 2.27566	valid_set's r2: 0.994107


	0.9943	 = Validation r2 score
	14.9s	 = Training runtime
	2.11s	 = Validation runtime
Fitting model: CatboostRegressor ...
	0.9948	 = Validation r2 score
	47.41s	 = Training runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetRegressor ...
	0.9944	 = Validation r2 score
	393.11s	 = Training runtime
	0.22s	 = Validation runtime
Fitting model: LightGBMRegressorCustom ...


[1000]	train_set's l2: 0.384033	train_set's r2: 0.998822	valid_set's l2: 2.00395	valid_set's r2: 0.994779


	0.9951	 = Validation r2 score
	18.47s	 = Training runtime
	1.78s	 = Validation runtime
Fitting model: weighted_ensemble_k0_l1 ...
	0.9953	 = Validation r2 score
	0.72s	 = Training runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 585.13s ...


In [ ]:
performance = predictor.evaluate(test)


Predictive performance on given dataset: r2 = 0.4974798672023729


In [ ]:
results = predictor.fit_summary()

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   weighted_ensemble_k0_l1   0.497480       0.077351  44.534966                0.001081           0.464597            1       True          9
1        NeuralNetRegressor   0.496933       0.059563  42.400362                0.059563          42.400362            0       True          7
2         LightGBMRegressor   0.312182       0.016708   1.670007                0.016708           1.670007            0       True          5
3         CatboostRegressor   0.296999       0.007457   4.279346                0.007457           4.279346            0       True          6
4   LightGBMRegressorCustom   0.292160       0.020781   1.811275                0.020781           1.811275            0       True          8
5  RandomForestRegressorMSE   0.268681       0.082410   5.898917                

In [ ]:
missers = data.loc[~data.index.isin(nomissings.index), :] 
#from the *data* df, pull those rows that 
#weren't included once we dropped rows that were missing MIN Payment values
predicted_MINIMUM_PAYMENTS = predictor.predict(missers)

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
missers['MINIMUM_PAYMENTS']=predicted_MINIMUM_PAYMENTS #plug-in interpreted values, obtained from autogluon

combined=nomissings.append(missers)
#recombine missers with nonmissers, creating a more robust dataset to model downstream analyses

/home/russell/anaconda3/envs/autoglu/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-18-a4e8d88483aa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missers['MINIMUM_PAYMENTS']=predicted_MINIMUM_PAYMENTS #plug-in interpreted values, obtained from autogluon


In [ ]:
combined

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.40,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.00,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.00,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.00,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12
5,C10006,1809.828751,1.000000,1333.28,0.00,1333.28,0.000000,0.666667,0.000000,0.583333,0.000000,0,8,1800.0,1400.057770,2407.246035,0.000000,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8919,C19160,14.524779,0.333333,152.00,152.00,0.00,0.000000,0.333333,0.333333,0.000000,0.000000,0,2,1500.0,0.000000,35.184391,0.000000,6
8929,C19170,371.527312,0.333333,0.00,0.00,0.00,1465.407927,0.000000,0.000000,0.000000,0.166667,5,0,1500.0,0.000000,132.262924,0.000000,6
8935,C19176,183.817004,1.000000,465.90,0.00,465.90,0.000000,1.000000,0.000000,0.833333,0.000000,0,6,1500.0,0.000000,350.997253,0.000000,6
8944,C19185,193.571722,0.833333,1012.73,1012.73,0.00,0.000000,0.333333,0.333333,0.000000,0.000000,0,2,4000.0,0.000000,70.037422,0.000000,6


In [ ]:
c_profile = ProfileReport(combined, title='Pandas Profiling Report', explorative=True)
c_profile.to_widgets()